In [10]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
#import seaborn
#seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
import matplotlib.ticker
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## Reading in the data

In [11]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,NaN,2023-05-04 10:56:50,0.0,-0.026053,2.270000e-09,2.500000e-09,4.996000e-09,4.452000e-07,-0.001018,317.0,-1.908,4266.874,-0.000458,7.340000e-07,NaN,NaN
1,NaN,2023-05-04 10:56:58,8.0,-0.023711,2.300000e-09,2.520000e-09,4.996000e-09,4.452000e-07,-0.001017,317.0,-1.908,4266.986,-0.000055,7.290000e-07,NaN,NaN
2,NaN,2023-05-04 10:57:07,17.0,-0.022131,2.290000e-09,2.510000e-09,4.996000e-09,4.452000e-07,-0.001024,317.0,-1.796,4267.211,-0.001293,7.240000e-07,NaN,NaN
3,NaN,2023-05-04 10:57:16,26.0,-0.021860,2.280000e-09,2.500000e-09,4.996000e-09,4.452000e-07,-0.001024,316.0,-1.683,4266.537,-0.001426,7.160000e-07,NaN,NaN
4,NaN,2023-05-04 10:57:24,34.0,-0.021860,2.280000e-09,2.510000e-09,4.996000e-09,4.452000e-07,-0.001027,316.0,-1.908,4267.099,-0.000834,7.150000e-07,NaN,NaN


In [12]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'closed turbo valve' 'injection volume prepared' 'injection complete'
 'emergency shutdown of instruments due to LHe refill pressure spike'
 'turning instruments on' 'refill complete' 'started ESD measurement'
 'esd measurement complete']


In [13]:
#Cut data from "transfer complete"
# Finding the index where the comment "transfer complete" was written
start_index = vaclog[vaclog["Live comments"] == 'refill complete'].index[0]

# Cutting the dataset from the start_index onwards
vaclog_cut = vaclog[start_index:].reset_index(drop=True)

# Printing unique comments in the "Live comments" column
print(pd.unique(vaclog_cut["Live comments"]))

['refill complete' nan 'started ESD measurement'
 'esd measurement complete']


### Temperature conversion 


In [14]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


### Trendline

In [15]:
## Checking helium level drop linearity
#defining a function for the straight line
def fit(x,a,b):
    return a*x + b

#curve fit for the data
y_val = vaclog_cut["helium"].values
x_val = (vaclog_cut["Elapsed time"]).values
params, cov = curve_fit(fit,x_val,y_val)

#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.4f}*x+{b:.2f}")
y_fit = fit(x_val, a, b)

b_second = 520
# Calculate the number of data points needed to reach b/a condition
num_points = int(abs(b / a) / (x_val[1] - x_val[0])) + 1
num_points2 = int(abs(b_second / a) / (x_val[1] - x_val[0])) + 1

# Extend the fitted line to y_fit = 0
x_fit_extended = np.linspace(x_val[0], (-b/a), num_points)
x_fit_extended2 = np.linspace(x_val[0], (-b_second/a), num_points2)

y_fit_extended = fit(x_fit_extended, a, b)
y_fit_extended2 = fit(x_fit_extended2, a, b_second)

print(x_fit_extended)
print(x_fit_extended2)


Fitted line: y = -0.0057*x+399.98
[16342.         16348.13554801 16354.27109602 ... 70089.40055935
 70095.53610736 70101.67165537]
[16342.         16348.56562381 16355.13124762 ... 91124.45518791
 91131.02081172 91137.58643553]


## Heat load    

In [26]:
L = 21 #J/g
rho = 125 #kg/m3

#reduction of LHe volume per second
r = 0.14 #m radius of LHe bath at full diam
l = abs(a)/1000 # reduction in LHe level per second
print(f"reduction in LHe level per second {l:.9f} m/s")
V  = 2*np.pi*r**2 *l #m3/s
print(f"Reduction of LHe volume per second {V:.9f} m3/s")
m = V*rho*1000 #g/s
print()
Q = L*m
print(f"The cryogenic heat load based on the LHe level reduction is {Q:.6f} W")


reduction in LHe level per second 0.000005706 m/s
Reduction of LHe volume per second 0.000000703 m3/s

The cryogenic heat load based on the LHe level reduction is 1.844468 W


### Plotting

In [17]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")

plt.rcParams["figure.figsize"] = [8,5]
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "black"
plt.rcParams["grid.linewidth"] = 0.35

In [18]:
# Helium level
fig, ax = plt.subplots()
ax.plot((vaclog_cut["Time"]), vaclog_cut["helium"], marker=".", markersize=5, label='inj volume pressure')
ax.plot((vaclog_cut["Time"]), y_fit, "--", color="red", label=f"Fitted line: y = {a:.4f}*x+{b:.2f}")
ax.legend(loc="upper right")
ax.set_xlabel('Timestamp')
ax.set_ylabel('LHe level (mm)')
ax.set_yscale('linear')
ax.get_yaxis().get_major_formatter().set_useOffset(False)
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

plt.title("Helium level drop and fit")
plt.savefig(r'./graphs/He_org_level.png', dpi=300,bbox_inches='tight')
plt.show()
    

In [19]:
#Fits
fig, ax = plt.subplots()
ax.plot(x_fit_extended/3600, y_fit_extended, "--", color="green", label=f"y = {a:.2f}*x+{b:.2f} extended fit")

ax.legend(loc="lower left")
ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('LHe level (mm)')
ax.set_yscale('linear')
# Set the number of major y ticks for ax
ax.yaxis.set_major_locator(plt.MaxNLocator(8))

ax.plot(x_fit_extended2/3600, y_fit_extended2, "--", color="orange", label=f"y = {a:.2f}*x+{b_second:.2f} extended fit")
ax.legend(loc="upper right")
ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('LHe level (mm)')
ax.set_yscale('linear')

# Set the number of major y ticks for ax2
ax.set_ylim(top=b_second)


plt.title("Helium level drop fit extrapolated to y=0")
plt.savefig(r'./graphs/He_fit_level.png', dpi=300,bbox_inches='tight')
plt.show()


In [20]:
""" #Helium level
fig, ax = plt.subplots()
y_fit = fit(x_val,a,b)
ax.plot((vaclog_cut["Time"]),vaclog_cut["helium"],marker=".", markersize=5,label='inj volume pressure')
ax.legend(loc="upper right")
ax.set_xlabel('Timestamp')
ax.set_ylabel('He-level (mm)')
ax.set_yscale('linear')
ax.get_yaxis().get_major_formatter().set_useOffset(False)
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

# Creating x values for prediction
x_pred = np.linspace(x_val[0], x_val[-1], num=100)

# Plotting the predicted line
y_pred = fit(x_pred, a, b)

ax2 = ax.twiny()
ax2.plot(vaclog_cut["Time"], y_pred, "--", color="red", label="Y = -ax + b fit")
ax2.legend(loc="lower left")
ax2.set_xlabel('Timestamp')
ax2.set_ylabel('He-level (mm)')
ax2.set_yscale('linear')
ax2.get_yaxis().get_major_formatter().set_useOffset(False)

plt.tight_layout()  # Ensure all elements are properly placed within the figure
plt.title("S1 Injection pressure evolution 1mbar conductance gauge")
plt.savefig(r'./graphs/He_level.png',dpi=300)
plt.show() """

' #Helium level\nfig, ax = plt.subplots()\ny_fit = fit(x_val,a,b)\nax.plot((vaclog_cut["Time"]),vaclog_cut["helium"],marker=".", markersize=5,label=\'inj volume pressure\')\nax.legend(loc="upper right")\nax.set_xlabel(\'Timestamp\')\nax.set_ylabel(\'He-level (mm)\')\nax.set_yscale(\'linear\')\nax.get_yaxis().get_major_formatter().set_useOffset(False)\nax.xaxis.set_major_formatter(md.DateFormatter(\'%H:%M\'))\nax.xaxis.set_major_locator(plt.MaxNLocator(10))\nfor label in ax.get_xticklabels(which=\'major\'):\n    label.set(rotation=30, horizontalalignment=\'right\')\n\n# Creating x values for prediction\nx_pred = np.linspace(x_val[0], x_val[-1], num=100)\n\n# Plotting the predicted line\ny_pred = fit(x_pred, a, b)\n\nax2 = ax.twiny()\nax2.plot(vaclog_cut["Time"], y_pred, "--", color="red", label="Y = -ax + b fit")\nax2.legend(loc="lower left")\nax2.set_xlabel(\'Timestamp\')\nax2.set_ylabel(\'He-level (mm)\')\nax2.set_yscale(\'linear\')\nax2.get_yaxis().get_major_formatter().set_useOffset

In [21]:
## Plotting the temp evolution
plt.figure()
plt.plot((vaclog_cut["Time"]),polyfit(A,vaclog_cut["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S1 Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/He_aut_Temp.png',dpi=300,bbox_inches='tight')
plt.show()